# Multiple Steps Time Series Forecasting with LSTM

**NOTE**: this notebook must be run using the kernel **Python 3.8 - Pytorch and Tensorflow**.

The implementation in this notebook is inspired to the example at https://machinelearningmastery.com/multi-step-time-series-forecasting-long-short-term-memory-networks-python/.

In [1]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
from matplotlib import patches as mpatches
import time
from scipy.optimize import minimize_scalar
from statsmodels.distributions.empirical_distribution import ECDF
from sklearn.model_selection import cross_validate, StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

import time
from collections import Counter
from joblib import Parallel, delayed
import sys
sys.path.append('../Modules')
from utils_bayesian_networks import *

In [2]:
# Specify sector, min date and max date
sector = 'Real Estate'
min_date = '2016-11-29'
max_date = '2020-03-31'

compute_partial_correlation = False

In [3]:
# Read input data to compute partial correlations

df_all, df_all_lookup, _, _, _ =\
    import_data_clustering_issuer_rf(issuer_curves_file_name='Basedati3_spread.csv',
                                    fx_rf_file_name='../Data/Observations - UniverseFX.csv',
                                    local_rf_file_name='../Data/Observations - UniverseLOCAL.csv',sep_issuer_curves_file=';')
df_all['CurveID_Short'] = df_all['CurveID'].apply(lambda x  : str(x).split('-')[0] + '_' + str(x).split('-')[1])
df_complete = pd.read_csv('../Data/conditional_volatility_issuers_BaseDati3.csv', sep=';')
df_complete['Date'] = pd.to_datetime(df_complete['Date'])
df_complete = df_complete.set_index('Date', drop=True)

In [4]:
# Select only curves from a specified sector
curves_sector = df_all[['CurveID', 'MS_Sector']].loc[lambda x: x['CurveID'].isin(df_complete.columns)].\
    drop_duplicates().\
    loc[lambda x: x['MS_Sector'] == sector]['CurveID'].unique()
print('Number of curves in sector ' + sector + ': ' + str(len(curves_sector)))

df_sector = df_complete.loc[:,curves_sector]
df_sector.index = pd.to_datetime(df_complete.index)
df_sector.shape

Number of curves in sector Real Estate: 34


(1352, 34)

In [5]:
# Rename columns in order to have same names in model graph and df_sector.columns

dict_mapping_names_short_complete = df_all[['CurveID', 'CurveID_Short']].drop_duplicates().set_index('CurveID').to_dict()['CurveID_Short']
df_sector.columns = [dict_mapping_names_short_complete[i] for i in list(df_sector.columns)]

In [6]:
df_sector_train = df_sector.loc[lambda x: x.index <= max_date]
df_sector_test = df_sector.loc[lambda x: x.index > max_date]

In [7]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


AttributeError: module 'tensorflow.compat.v2' has no attribute '__internal__'

In [ ]:
target = 'KWELN_3275897'
num_reg_local = 5

df_sector_train = df_sector_train.asfreq(freq='1d', method='bfill')
df_sector_test = df_sector_test.asfreq(freq='1d', method='bfill')

regressors_local = [i for i in df_sector.columns if i != target]
reg_local_to_include = regressors_local[:num_reg_local]

# exog_train = df_sector_train[reg_local_to_include]
# exog_test = df_sector_test[reg_local_to_include]
y_train = df_sector_train[target]
y_test = df_sector_test[target]

In [ ]:
# Concatenate train and test dataframes
df_train_test = pd.concat([df_sector_train, df_sector_test])[[*[target], *reg_local_to_include]]

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [df.columns[j] + ('(t-%d)' % i) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [df.columns[j] + '(t)' for j in range(n_vars)]
        else:
            names += [df.columns[j] + ('(t+%d)' % i) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
##### PARAMETERS

# Number of steps to predict
num_steps_to_predict = 4

# Number of lag to consider
n_lag = 1

# Number of predictors (on top of lagged target)
n_pred = len(reg_local_to_include)

# Train test size
train_size = 1700

# Neural network parameters
n_batch=1
nb_epoch=10
n_neurons=1

In [ ]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = pd.DataFrame(scaler.fit_transform(df_train_test), columns=df_train_test.columns)
# frame as supervised learning
reframed = series_to_supervised(scaled, n_out=num_steps_to_predict)
# drop columns we don't want to predict
reframed.drop(columns=[i + '(t)' for i in reg_local_to_include], inplace=True)
reframed.drop(columns=[i + '(t+' + str(k) + ')' for i in reg_local_to_include for k in range(1,4)], inplace=True)
print(reframed.head())

In [ ]:
# Create train and test sets

train = reframed.values[:train_size, :]
test = reframed.values[train_size:, :]

In [ ]:
# Fit LSTM model

X, y = train[:, 0:(n_lag*(n_pred+1))], train[:, (n_lag*(n_pred+1)):]
X = X.reshape(X.shape[0], 1, X.shape[1])
# design network
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(y.shape[1]))
model.compile(loss='mean_squared_error', optimizer='adam')
# fit network
for i in range(nb_epoch):
    model.fit(X, y, epochs=1, batch_size=n_batch, verbose=True, shuffle=False)
    model.reset_states()

In [ ]:
# Make predictions using fitted LSTM

def forecast_lstm(model, X, n_batch):
    # reshape input pattern to [samples, timesteps, features]
    X = X.reshape(1, 1, len(X))
    # make forecast
    forecast = model.predict(X, batch_size=n_batch)
    # convert to array
    return [x for x in forecast[0, :]]

forecasts = list()
for i in range(len(test)):
    X, y = test[i, 0:(n_lag*(n_pred+1))], test[i, (n_lag*(n_pred+1)):]
    # make forecast
    forecast = forecast_lstm(model, X, n_batch)
    # store the forecast
    forecasts.append(forecast)

In [ ]:
# evaluate the RMSE for each forecast time step
def evaluate_forecasts(test, forecasts, n_lag, n_seq):
    for i in range(n_seq):
        actual = [row[i] for row in test]
        predicted = [forecast[i] for forecast in forecasts]
        rmse = sqrt(mean_squared_error(actual, predicted))
        print('t+%d RMSE: %f' % ((i+1), rmse))

# plot the forecasts in the context of the original dataset
def plot_forecasts(series, forecasts, n_test):
    # plot the entire dataset in blue
    pyplot.plot(series.values)
    # plot the forecasts in red
    for i in range(len(forecasts)):
        off_s = len(series) - n_test + i - 1
        off_e = off_s + len(forecasts[i]) + 1
        xaxis = [x for x in range(off_s, off_e)]
        yaxis = [series.values[off_s]] + forecasts[i]
        pyplot.plot(xaxis, yaxis, color='red')
    # show the plot
    pyplot.show()
    
    
actual = [row[(n_lag*(n_pred+1)):] for row in test]
evaluate_forecasts(actual, forecasts, n_lag, num_steps_to_predict)


plot_forecasts(scaled[target], forecasts, len(forecasts)+2)

**NOTE**: we neede to understand how to define the final forecast at a certain time step t, by combining the different predictions referred to that time step on different rows of the forecasts df.